# 1. Install and import dependencies 

In [ ]:
# install in command prompt
# !pip install tensorflow==2.9.1 tensorflow-gpu==2.9.1 opencv-python mediapipe sklearn matplotlib

In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic 

In [2]:
mp_holistic = mp.solutions.holistic  # Model to make detections
mp_drawing = mp.solutions.drawing_utils  # Tools to draw landmarks?

In [3]:
# Detect the landmarks given an image / a frame
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # converts BGR image (read by cv2) to RGB image to be processed by holistic model
    image.flags.writeable = False  # this helps save some memory 
    results = model.process(image)  # make prediction using holistic model
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # coverts RGB image back to BGR image
    return image, results

In [4]:
# Render the landmarks stored in results on the image
def draw_landmarks(image, results):
    # draw face landmarks in red, face connections in green
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(0,0,255), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(0,255,0), thickness=1, circle_radius=1))
    # draw pose landmarks in yellow, pose connections in purple
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(0,255,255), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(255,0,255), thickness=1, circle_radius=1))
    # draw hand landmarks in orange, hand connections in blue
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(0,165,255), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(255,0,0), thickness=1, circle_radius=1))
    # draw hand landmarks in orange, hand connections in blue
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(0,165,255), thickness=2, circle_radius=4),
                             mp_drawing.DrawingSpec(color=(255,0,0), thickness=1, circle_radius=1))

In [5]:
# Mediapipe loop, to reuse in later parts of the code

cap = cv2.VideoCapture(0)  # access webcam, that is video device 0
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():  # looping through all frames in webcam
        # Read feed
        ret, frame = cap.read()

        # Make prediction
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_landmarks(image, results)

        # Show to screen
        cv2.imshow("OpenCV feed", image)

        # Break out of loop if keyboard 'q' is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
# display processed last frame
draw_landmarks(frame, results)
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

# 3. Extract Keypoint Values 

In [7]:
# to use extracted keypoint values to decode the sign language detected
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

    # print(pose.shape)
    # print(face.shape)
    # print(lh.shape)
    # print(rh.shape)
    
    return np.concatenate([pose, face, lh, rh])

# 4. Setup Folders for Collection 

In [8]:
# Customize these variables for different use cases

# Path for exported data, which are keypoints that we will extract
DATA_PATH = os.path.join('MP_DATA')

# Actions to detect
actions = np.array(['i', 'love', 'code'])

# Number of videos worth of data
no_sequences = 30

# Number of frames for each video
sequence_length = 30

In [9]:
# Creates folders MP_DATA/action/sequence
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

# 5. Collect Keypoint Values for Training and Testing 

## IMPORTANT WHEN TRAINING
- Good to use actions that are vastly different
- Big actions might work better i.e. move hands more exaggeratedly
- To retrain, remove "Logs" and "MP_DATA" folders (if they exist)

In [10]:
# Mediapipe loop, to reuse in later parts of the code

cap = cv2.VideoCapture(0)  # access webcam, that is video device 0
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # ['hello', 'thanks', 'iloveyou'...]
    for action in actions:
        # looping through videos
        for sequence in range(no_sequences):
            # looping through frames in video
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make prediction
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_landmarks(image, results)

                # Collect videos, allow for breaks before next video starts, to give time to get into position
                if frame_num == 0:
                    cv2.putText(image, 'STARTING COLLECTION', (120, 200),
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'COLLECTING FRAMES FOR {} Video number {}'.format(action, sequence), (15, 12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    cv2.waitKey(2000)
                else:
                    cv2.putText(image, 'COLLECTING FRAMES FOR {} Video number {}'.format(action, sequence), (15, 12),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                
                    # Show to screen
                    cv2.imshow("OpenCV feed", image)
                
                # Export keypoint values
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)
                
                # Break out of loop if keyboard 'q' is pressed
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
# Uncomment and run if camera is still on
# cap.release()
# cv2.destroyAllWindows()

# 6. Preprocess Data and Create Labels and Features

In [11]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [12]:
label_map = {label:num for num, label in enumerate(actions)}

In [13]:
# Read in data from all the numpy arrays recorded in step 5
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [14]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)  # one-hot encoding

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [16]:
print(X_train.shape)
print(X_test.shape)

(85, 30, 1662)
(5, 30, 1662)


#  7. Build and Train LSTM Neural Network

In [17]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [31]:
# To monitor training 
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [41]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [42]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [34]:
# Maybe stop earlier, might be good to stop at 300 epochs or whenever loss ~0.1, categorical_accuracy ~0.9
model.fit(X_train, y_train, epochs=1000, callbacks=[tb_callback])  

# run command below in command prompt to track progress
# cd <path to Logs/train>
# tensorboard --logdir=.
# go to local host address given in command prompt

Epoch 1/500
3/3 [==============================] - 3s 260ms/step - loss: 2.1917 - categorical_accuracy: 0.3647
Epoch 2/500
3/3 [==============================] - 1s 260ms/step - loss: 1.3007 - categorical_accuracy: 0.3176
Epoch 3/500
3/3 [==============================] - 1s 254ms/step - loss: 1.1027 - categorical_accuracy: 0.2824
Epoch 4/500
3/3 [==============================] - 1s 261ms/step - loss: 1.1379 - categorical_accuracy: 0.4353
Epoch 5/500
3/3 [==============================] - 1s 255ms/step - loss: 1.1844 - categorical_accuracy: 0.2941
Epoch 6/500
3/3 [==============================] - 1s 349ms/step - loss: 1.0910 - categorical_accuracy: 0.3647
Epoch 7/500
3/3 [==============================] - 1s 307ms/step - loss: 1.0142 - categorical_accuracy: 0.4706
Epoch 8/500
3/3 [==============================] - 1s 298ms/step - loss: 1.2145 - categorical_accuracy: 0.3882
Epoch 9/500
3/3 [==============================] - 1s 295ms/step - loss: 1.1236 - categorical_accuracy: 0.4118
E

KeyboardInterrupt: 

In [35]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 3)                 99        
                                                                 
Total params: 596,675
Trainable params: 596,675
Non-tr

# 8. Make Predictions

In [36]:
res = model.predict(X_test)

1/1 [==============================] - 0s 362ms/step


In [37]:
idx = 3
print(actions[np.argmax(res[idx])])
print(actions[np.argmax(y_test[idx])])

love
love


# 9. Save or Load Weights 

In [38]:
model.save('action.h5')

In [39]:
# delete current model, need to recompile model
# del model

In [43]:
# load weights, make sure model has been compiled (Step 7)
# model.load_weights('action.h5')

# 10. Evaluation Using Confusion Matrix and Accuracy

In [44]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [45]:
yhat = model.predict(X_test)

1/1 [==============================] - 0s 331ms/step


In [46]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [47]:
# We want the numbers on top-left and bottom-right (true positives and true negatives)
multilabel_confusion_matrix(ytrue, yhat)

array([[[4, 1],
        [0, 0]],

       [[2, 0],
        [1, 2]],

       [[3, 0],
        [0, 2]]], dtype=int64)

In [48]:
# Higher score is better
accuracy_score(ytrue, yhat)

0.8

# 11. Test in Real Time 

In [49]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [50]:
# Mediapipe loop, to reuse in later parts of the code

sequence = []  # collect 30 frames in this list, pass into model to predict action
sentence = []  # concatenate history of detections
predictions = []  # improve stability
threshold = 0.7

cap = cv2.VideoCapture(0)  # access webcam, that is video device 0
# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():  # looping through all frames in webcam
        # Read feed
        ret, frame = cap.read()

        # Make prediction
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_landmarks(image, results)
        
        # Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]  # Collect last 30 frames into sequence
        
        if len(sequence) == 30:
            # Only start predicting and visualizing when there are 30 frames
            res = model.predict(np.expand_dims(sequence, axis=0))[0]  # change shape of sequence to (1, 30, 1662) instead of (30, 1662)
            predictions.append(np.argmax(res))
            
            # Visualization 
            if np.unique(predictions[-10:])[0] == np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            # Only append if a new word is detected
                            sentence.append(actions[np.argmax(res)])
                    else:
                        # Current sentence is empty, just append
                        sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5:
                sentence = sentence[-5:]

            # Visualize probabilities
            image = prob_viz(res, actions, image, colors)

            cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
            cv2.putText(image, ' '.join(sentence), (3, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            

        # Show to screen
        cv2.imshow("OpenCV feed", image)

        # Break out of loop if keyboard 'q' is pressed
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 31ms/step


1/1 [==============================] - 0s 31ms/step
